# NOTE

*We have recently experienced issues with `azureml.contrib.brainwave` library. Till the issues are addressed we are suspending development of FPGA labs. These are the labs with **1x** prefix*.


# FPGA Deployment

In this lab you will deploy the custom image classification model developed in Lab 1 to an FPGA powered cloud services.

Project Brainwave is a hardware architecture from Microsoft. It's based on Intel's FPGA devices, which data scientists and developers use to accelerate real-time AI calculations. This FPGA-enabled architecture offers performance, flexibility, and scale, and is available on Azure.

FPGAs make it possible to achieve low latency for real-time inferencing requests. Asynchronous requests (batching) aren't needed. Batching can cause latency, because more data needs to be processed. Project Brainwave implementations of neural processing units don't require batching; therefore the latency can be many times lower, compared to CPU and GPU processors.

You can reconfigure FPGAs for different types of machine learning models. This flexibility makes it easier to accelerate the applications based on the most optimal numerical precision and memory model being used. Because FPGAs are reconfigurable, you can stay current with the requirements of rapidly changing AI algorithms.

Today, Project Brainwave supports:

- Image classification and recognition scenarios
- TensorFlow deployment
- DNNs: ResNet 50, ResNet 152, VGG-16, SSD-VGG, and DenseNet-121
- Intel FPGA hardware

Using this FPGA-enabled hardware architecture, trained neural networks run quickly and with lower latency. Project Brainwave can parallelize pre-trained deep neural networks (DNN) across FPGAs to scale out your service. The DNNs can be pre-trained, as a deep featurizer for transfer learning, or fine-tuned with updated weights.

To deploy FPGA service you need to:
- Create a service definition
- Register the model 
- Deploy the service with the registered model


# Note

*Brainwave* requires TensorFlow 1.10. Currently, DSVM includes TensorFlow 1.12. To run the lab you need to downgrade TensorFlow in `py36` environment on DSVM.

### Connect to AML Workspace

In [ ]:
# Check core SDK version number
import azureml.core
import os
print("SDK version:", azureml.core.VERSION)

In [ ]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Deploy the model

### Create a service definition

A service definition is a file describing a pipeline of graphs (input, featurizer, and classifier) based on TensorFlow. The deployment command automatically compresses the definition and graphs into a ZIP file, and uploads the ZIP to Azure Blob storage. The DNN is already deployed on Project Brainwave to run on the FPGA.

Here we use the TF.Keras model trained in the lab 1 in the classifier stage.



### Retrieve the top model from the model registry

In [ ]:
from azureml.core.model import Model
from keras.models import load_model
import tensorflow as tf


# Retrieve the model file
model_name = 'aerial_classifier_brainwave'
model_path = Model.get_model_path(model_name, _workspace=ws)

# Rehydrate the model
model = load_model(model_path)
model.summary()


### Define a service

Define input tensors node.

In [ ]:
import azureml.contrib.brainwave.models.utils as utils
in_images = tf.placeholder(tf.string)
image_tensors = utils.preprocess_array(in_images)
print(image_tensors.shape)


Define **resnet50** featurizer. It has to be the same model as used in Lab1 to create bottleneck features.


In [ ]:
from azureml.contrib.brainwave.models import QuantizedResnet50
model_path = os.path.expanduser('~/models')
bwmodel = QuantizedResnet50(model_path, is_frozen = True)
print(bwmodel.version)

Define the FPGA pipeline

In [ ]:
from azureml.contrib.brainwave.pipeline import ModelDefinition, TensorflowStage, BrainWaveStage, KerasStage

model_def = ModelDefinition()
model_def.pipeline.append(TensorflowStage(tf.Session(), in_images, image_tensors))
model_def.pipeline.append(BrainWaveStage(tf.Session(), bwmodel))
model_def.pipeline.append(KerasStage(model))

model_def_path = './model_def'
model_def.save(model_def_path)
print(model_def_path)

### Deploy

Register with model registry

In [ ]:
from azureml.core.model import Model
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')
model_name = "aerial-classifier-brainwave"
service_name = "aerial-brainwave-service"

registered_model = Model.register(ws, model_def_path, model_name)

Deploy.

In [ ]:
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.contrib.brainwave import BrainwaveWebservice, BrainwaveImage
try:
    service = Webservice(ws, service_name)
except WebserviceException:
    image_config = BrainwaveImage.image_configuration()
    deployment_config = BrainwaveWebservice.deploy_configuration()
    service = Webservice.deploy_from_model(ws, service_name, [registered_model], image_config, deployment_config)
    service.wait_for_deployment(True)

## Test the service

The *Brainwave* library includes a simple client that can be used for testing.

In [ ]:
from azureml.contrib.brainwave.client import PredictionClient
client = PredictionClient(service.ipAddress, service.port)

In [ ]:
print(service.ipAddress + ':' + str(service.port))